this is to poc retrieval only. no document prepartion.

this is for POC purpose. it ensures that when only partial code that retrieve document is run. it can be run successfully. in Aj. code, full opensearch initialization, corpus database building, uploading, embedding, and retrieval are performed. we are going to use only retrieval part. it may need some modification.

POC result: ok with some modification in vector database, and index part.

this code is used in llm-infer-rag.py POC for fully POC in deployment environment.

note: it run successfully. but in one execution run, it fail because of huggingface. this is likely because huggingface limitation or free account limitation, or perhaps model maintenance.

In [1]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
import nest_asyncio
from os import getenv

#4. Generate embeddings using the BAAI/bge-m3 model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#5.1 Set up OpenSearch connection details
#this step is mandatory so that the embeding / vector can be stored in the opensearch.
from os import getenv
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

from llama_index.core import VectorStoreIndex, StorageContext

from llama_index.core.vector_stores.types import VectorStoreQueryMode


#from fastapi import FastAPI
from pydantic import BaseModel

import ollama

/Volumes/SS2305_1MH01/anaconda-env/kdai-llm-final/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#do not execute this part if testing the logic and huggingface.
# Create a FastAPI instance
#app = FastAPI()

In [2]:
# Define the input data model
class TextInput(BaseModel):
    text: str

In [3]:
def query_openthaigpt(question, context):
    formatted_prompt = f'''# Few-shot examples
Example 1:
Context: Mr. A killed a person and claimed it was justifiable self-defense under Section 68 of the Criminal Code.
If the facts indicate that there was a struggle for a firearm and the gun accidentally discharged.
Question: How would the court rule? Please provide an example Supreme Court decision case number or the Red case number (หมายเลขคดีแดง)
Answer: The court would likely determine that the defendant's claim of self-defense is not tenable. Justifiable self-defense under Section 68 requires intentional action. Given that the facts indicate a struggle for the firearm and an accidental discharge, this constitutes a negligent act resulting in death, not an intentional act. Therefore, it does not meet the criteria for justifiable self-defense under the law.
The court would likely rule that this case falls under negligent homicide rather than justifiable self-defense,
as the element of intent, which is crucial for the self-defense claim,
is absent in the scenario where the firearm discharged accidentally during a struggle.
The example Supreme Court decision is Case No. 1597/2562, Red Case No. อ632/2560.

# RAG component
Retrieved information:
{context}

# Actual question
Question: {question}

Please answer the question with Thai language using the information from the examples and the retrieved information above. Focus on troubleshooting end-user devices in FTTX projects. Provide a clear, step-by-step answer, prioritized in order of importance. Include a brief explanation for each step. If the provided information is insufficient to answer the question completely, state "The available information is not sufficient to fully answer this question" and suggest general troubleshooting steps.

Answer:'''

    #print(formatted_prompt)
    #response = ollama.generate(model='openthaigpt:latest', prompt=formatted_prompt)
    response = ollama.generate(model='llama3.2:latest', prompt=formatted_prompt)
    return response['response']

In [4]:
# Define the function to process the text
def process_input_text(input_text: str) -> str:

    # Check if CUDA is available for GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    #from llama_index.embeddings.huggingface import HuggingFaceEmbedding
    embedding_model_name = 'BAAI/bge-m3'
    embedding_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=512, device=device)

    embeddings = embedding_model.get_text_embedding("box")
    dim = len(embeddings)

    #endpoint = getenv("OPENSEARCH_ENDPOINT", "http://localhost:9200")
    endpoint = getenv("OPENSEARCH_ENDPOINT", "http://192.168.50.204:9200") #host machine ip address
    #index to demonstrate the VectorStore impl
    idx = getenv("OPENSEARCH_INDEX", "test_pdf_index")

    #5.2 Configure OpenSearchVectorClient
    # OpensearchVectorClient stores text in this field by default
    text_field = "content_text"
    # OpensearchVectorClient stores embeddings in this field by default
    embedding_field = "embedding"
    # OpensearchVectorClient encapsulates logic for a
    # single opensearch index with vector search enabled with hybrid search pipeline
    client = OpensearchVectorClient(
        endpoint=endpoint,
        index=idx,
        dim=dim,
        embedding_field=embedding_field,
        text_field=text_field,
        search_pipeline="hybrid-search-pipeline",
    )

    # initialize vector store
    vector_store = OpensearchVectorStore(client)

    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    #index = VectorStoreIndex(
    #    storage_context=storage_context, embed_model=embedding_model
    #)

    # No need for token_nodes or any other objects since you're using OpenSearch
    #index_struct = None  # If you have an existing index structure, load it here
    #index = VectorStoreIndex.from_vector_store(
    #    storage_context=storage_context,
    #    embed_model=embedding_model,
    #    index_struct=index_struct,  # Can be None if you are relying on vector store fully
    #)

    # Initialize the VectorStoreIndex with the vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,  # Pass the vector store you created earlier
        storage_context=storage_context,
        embed_model=embedding_model
    )

    retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)

    question = input_text

    prompt = retriever.retrieve(question)

    #answer = answer_question(question)
    print(question)
    print(prompt[0])
    answer = query_openthaigpt(question, prompt[0])
    print("answer:",answer)

    return answer

In [5]:
#processed_text = process_input_text(input_data.text)
input_text = 'กระทำการโดยประมาทเป็นเหตุให้ถึงแก่ความตาย ป.อ.291 ไม่ได้บรรยายฟ้องถึงพฤติการณ์ ศาลจะพิพากษาลงโทษจำเลยได้หรือไม่'
processed_text = process_input_text(input_text)

Using device: cpu
กระทำการโดยประมาทเป็นเหตุให้ถึงแก่ความตาย ป.อ.291 ไม่ได้บรรยายฟ้องถึงพฤติการณ์ ศาลจะพิพากษาลงโทษจำเลยได้หรือไม่
Node ID: 578b2dd8-d2fb-4642-8668-aa86b57ba3aa
Text: ศาล ทธร ภาค  1   ลงโทษตาม อเ จจ ง ไ ความตามทาง จารณาไ 
เพราะการกระ โดยเจตนาและ การกระ โดยประมาทไ ใ  อแตก างในสาระ  ญ
และไ  อเ นเ องเ น ขอห อ
เ นเ อง โจท ไ ประสง ใ ลงโทษตามประมวลกฎหมาย   จารณาความอาญา มาตรา  192
วรรคสองและวรรคสาม   ศาล ทธร ภาค  1  พากษาลงโทษ เลย   3 ฐาน
ประมาทเ นเห ใ   น งแ ความตายตามประมวลกฎหมายอาญา  มาตรา  291  งชอบ แ ว
  ญหา...
Score:  1.000

answer: การวิเคราะห์ข้อมูลที่ได้รับ:

1. ข้อความตัวอย่างที่ 1 บ่งชี้ว่าในกรณีที่มีการต่อสู้เพื่อกระสุนปืนและกระสุนปืนไกลลอยไปกระแทกผู้คน ส่วนใหญ่จะพิจารณาว่าเป็นเหตุผลสำหรับการประณามตามกฎหมาย ไม่ใช่การทำร้ายทางใจ
2. ข้อความที่ได้รับจาก RAG บ่งชี้ว่าในกรณีที่มีการกระทำโดยประมาททำให้เกิดการเสียชีวิต ส่วนใหญ่จะพิจารณาในฐานะการประณามตามประมวลกฎหมายอาญา มาตรา 291
3. ข้อความตัวอย่างที่ 1 ยังระบุว่าในกรณีที่มีการต่อสู้เพื่อกระสุนปืนและกระสุนปืนไกลลอยไปกระแท

In [6]:
print(processed_text)

การวิเคราะห์ข้อมูลที่ได้รับ:

1. ข้อความตัวอย่างที่ 1 บ่งชี้ว่าในกรณีที่มีการต่อสู้เพื่อกระสุนปืนและกระสุนปืนไกลลอยไปกระแทกผู้คน ส่วนใหญ่จะพิจารณาว่าเป็นเหตุผลสำหรับการประณามตามกฎหมาย ไม่ใช่การทำร้ายทางใจ
2. ข้อความที่ได้รับจาก RAG บ่งชี้ว่าในกรณีที่มีการกระทำโดยประมาททำให้เกิดการเสียชีวิต ส่วนใหญ่จะพิจารณาในฐานะการประณามตามประมวลกฎหมายอาญา มาตรา 291
3. ข้อความตัวอย่างที่ 1 ยังระบุว่าในกรณีที่มีการต่อสู้เพื่อกระสุนปืนและกระสุนปืนไกลลอยไปกระแทกผู้คน ส่วนใหญ่จะพิจารณาว่าเป็นการประณามตามประมวลกฎหมายอาญา มาตรา 291 แทนการประณามตามประมวลกฎหมายอาญา มาตรา 192

คำตอบ:

คำถามคือ "ว่ากระทำการโดยประมาทเป็นเหตุให้ถึงแก่ความตาย ป.อ. 291 จะได้รับการพิพากษาลงโทษจำเลยหรือไม่"

ข้อสรุป: ในกรณีที่มีการกระทำโดยประมาททำให้เกิดการเสียชีวิต ส่วนใหญ่จะพิจารณาในฐานะการประณามตามประมวลกฎหมายอาญา มาตรา 291 แทนการประณามตามประมวลกฎหมายอาญา มาตรา 192 ดังนั้น กระทำการโดยประมาทเป็นเหตุให้ถึงแก่ความตาย ป.อ. 291 จะได้รับการพิพากษาลงโทษจำเลย

คำแนะนำเพิ่มเติม: หากมีข้อมูลเพิ่มเติมเกี่ยวกับกรณีนี้ เช่น การต่อสู้เพื่อกระสุ

In [7]:
#processed_text = process_input_text(input_data.text)
input_text = 'กระทำการโดยประมาทเป็นเหตุให้ถึงแก่ความตาย ป.อ.291 ไม่ได้บรรยายฟ้องถึงพฤติการณ์ ศาลจะพิพากษาลงโทษจำเลยได้หรือไม่'
processed_text = process_input_text(input_text)

Using device: cpu
กระทำการโดยประมาทเป็นเหตุให้ถึงแก่ความตาย ป.อ.291 ไม่ได้บรรยายฟ้องถึงพฤติการณ์ ศาลจะพิพากษาลงโทษจำเลยได้หรือไม่
Node ID: 578b2dd8-d2fb-4642-8668-aa86b57ba3aa
Text: ศาล ทธร ภาค  1   ลงโทษตาม อเ จจ ง ไ ความตามทาง จารณาไ 
เพราะการกระ โดยเจตนาและ การกระ โดยประมาทไ ใ  อแตก างในสาระ  ญ
และไ  อเ นเ องเ น ขอห อ
เ นเ อง โจท ไ ประสง ใ ลงโทษตามประมวลกฎหมาย   จารณาความอาญา มาตรา  192
วรรคสองและวรรคสาม   ศาล ทธร ภาค  1  พากษาลงโทษ เลย   3 ฐาน
ประมาทเ นเห ใ   น งแ ความตายตามประมวลกฎหมายอาญา  มาตรา  291  งชอบ แ ว
  ญหา...
Score:  1.000

answer: การประเมินสถานการณ์ของคดีละต้า

1. การตรวจสอบข้อเท็จจริง: ศาลจะต้องตรวจสอบข้อเท็จจริงของการเหตุการณ์ที่เกิดขึ้น รวมถึงการกระทำของผู้จำเลยและผลที่ตามมา
2. การวิเคราะห์ความสัมพันธ์กับมาตรา 291 ของประมวลกฎหมายอาญา: ศาลจะต้องตรวจสอบว่าการกระทำของผู้จำเลยมีส่วนเกี่ยวข้องกับการเสียชีวิตหรือไม่ และมีให้การสนับสนุนหรือไม่
3. การพิจารณาตามมาตรา 192 ของประมวลกฎหมายอาญา: ศาลจะต้องพิจารณาว่าการกระทำของผู้จำเลยเป็นการกระทำโดยเจตนาหรือจากการประมาท และมีส่ว